In [ ]:
import numpy as np
import random

import pandas as pd
from scipy import stats

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

In [ ]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)

In [ ]:
train_data=pd.read_csv("train.csv")
train_data.head()

In [ ]:
test_data=pd.read_csv("test.csv")

In [ ]:
# Count missing values for each column
missing_values = train_data.isnull().sum()

# Create a DataFrame to store the count of missing values
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'MissingCount': missing_values.values
})

# Add a column to show the percentage of missing values
missing_df['MissingPercentage'] = (missing_df['MissingCount'] / len(train_data)) * 100

# Sort the DataFrame by the number of missing values in descending order
missing_df.sort_values(by='MissingCount', ascending=False, inplace=True)

# Reset index for readability
missing_df.reset_index(drop=True, inplace=True)

# Display the DataFrame
print(missing_df)

- null rate for dropping set to > 60%.
- Try 30% and 10%

In [ ]:
# Drop columns with missing percentage greater than 60%
columns_to_drop = missing_df[missing_df['MissingPercentage'] > 10]['Column'].tolist()

# Drop the identified columns from the DataFrame
train_data.drop(columns=columns_to_drop, axis=1, inplace=True)

# Display the updated DataFrame shape after dropping columns
print(f"Updated shape of the DataFrame: {train_data.shape}")

In [ ]:
# drop same columns from test data
test_data.drop(columns=columns_to_drop, axis=1, inplace=True)

# Display the updated DataFrame shape after dropping columns
print(f"Updated shape of the DataFrame: {test_data.shape}")

In [ ]:
# Set the 'UID' column as the index
train_data.set_index('UID', inplace=True)

# Display the updated DataFrame to confirm the change
# print(train_data.head())
train_data.head()

In [ ]:
# Set the 'UID' column as the index
test_data.set_index('UID', inplace=True)

# Display the updated DataFrame to confirm the change
# print(test_data.head())
test_data.head()

In [ ]:
# Define the mapping for 'Target' column
target_mapping = {'low': 0, 'medium': 1, 'high': 2}

# Apply the mapping to the 'Target' column
train_labels = train_data['Target'].map(target_mapping)

# Display the first few rows of the labels to verify the mapping
print(train_labels.head())

In [ ]:
train_data = train_data.drop(columns=['TownId','Target','DistrictId'])

test_data = test_data.drop(columns=['TownId','DistrictId'])

In [ ]:
def fill_missing_values(df):
    # Define the columns based on their type
    categorical_columns = [
        'HarvestProcessingType', 'SoilFertilityType', 'AgricultureZoningCode',
        'ValuationYear', 'NationalRegionCode', 'StorageAndFacilityCount', 'RawLocationId',
        'LandUsageType', 'CropSpeciesVariety', 'AgriculturalPostalZone'
    ]
    
    median_columns = [
        'FarmingUnitCount', 'FieldSizeSqft', 'CultivatedAreaSqft1', 'MainIrrigationSystemCount',
        'FieldEstablishedYear', 'TotalTaxAssessed', 'TaxLandValue', 'TotalCultivatedAreaSqft',
        'WaterAccessPoints', 'TaxAgrarianValue', 'TotalValue'
    ]
    
    mean_columns = [
        'WaterAccessPointsCalc', 'Longitude', 'Latitude'
    ]
    
    # Convert categorical columns to 'object' type if necessary
    for column in categorical_columns:
        if column in df.columns:
            df[column] = df[column].astype('object')

    # Fill missing values for categorical columns using mode
    for column in categorical_columns:
        if column in df.columns:
            if df[column].isnull().sum() > 0:
                try:
                    mode_value = df[column].mode(dropna=True)[0] if not df[column].mode().empty else None
                    if mode_value is not None:
                        df[column].fillna(mode_value, inplace=True)
                    else:
                        print(f"Warning: Could not find a mode for column {column}")
                except Exception as e:
                    print(f"Error while filling mode for column {column}: {e}")
    
    # Fill missing values for numerical columns using median
    for column in median_columns:
        if column in df.columns and df[column].dtype in ['int64', 'float64']:
            if df[column].isnull().sum() > 0:
                median_value = df[column].median()
                df[column].fillna(median_value, inplace=True)
    
    # Fill missing values for numerical columns using mean
    for column in mean_columns:
        if column in df.columns and df[column].dtype in ['int64', 'float64']:
            if df[column].isnull().sum() > 0:
                mean_value = df[column].mean()
                df[column].fillna(mean_value, inplace=True)
    
    return df

# Fill missing values in the training data
train_data = fill_missing_values(train_data)

# Check if there are still missing values
missing_values = train_data.isnull().sum()
print("Missing values after filling:\n", missing_values[missing_values > 0])


In [ ]:
# fill missing values in test data
test_data = fill_missing_values(test_data)

# Check if there are still missing values
missing_values = test_data.isnull().sum()

In [ ]:
# get feature importance using random forest
from sklearn.ensemble import RandomForestClassifier

# Separate the features and target variable
X = train_data
y = train_labels

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=seed)

# Fit the model
rf.fit(X, y)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to store the feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by feature importance in descending order
feature_importances_df.sort_values(by='Importance', ascending=False, inplace=True)

# Reset index for readability
feature_importances_df.reset_index(drop=True, inplace=True)

# Display the DataFrame
print(feature_importances_df)

In [ ]:
# drop columns with importance less than 0.01
columns_to_drop = feature_importances_df[feature_importances_df['Importance'] < 0.015]['Feature'].tolist()

# Drop the identified columns from the DataFrame
train_data.drop(columns=columns_to_drop, axis=1, inplace=True)

# Drop the identified columns from test data
test_data.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
# print count of unique values in y
print(y.value_counts())

In [ ]:
train_data.info()

In [ ]:
# convert Agriculturepostalzone to int
train_data['AgriculturalPostalZone'] = train_data['AgriculturalPostalZone'].astype(int)
test_data['AgriculturalPostalZone'] = test_data['AgriculturalPostalZone'].astype(int)

In [ ]:
# add uid to train and test data and save cleaned data
# train_data['UID'] = train_data.index
# test_data['UID'] = test_data.index

# train_data['Target'] = train_labels

# train_data.to_csv("cleaned_train.csv", index=False)
# test_data.to_csv("cleaned_test.csv", index=False)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize the features before training
scaler = StandardScaler()
train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns, index=train_data.index)
test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns, index=test_data.index)

# train_data_norm = scaler.fit_transform(train_data)
# test_data_norm = scaler.fit_transform(test_data)

In [ ]:
# make train data with 22514 data of each class

# Separate the data based on the target classes
low_class = train_data[train_labels == 0]
medium_class = train_data[train_labels == 1]
high_class = train_data[train_labels == 2]

# Get the number of samples in each class
low_class_count = len(low_class)
medium_class_count = len(medium_class)
high_class_count = len(high_class)

# Set the number of samples to be selected from each class
num_samples = min(low_class_count, medium_class_count, high_class_count)

# Randomly sample data from each class
low_class_sample = low_class.sample(n=num_samples, random_state=seed)
medium_class_sample = medium_class.sample(n=num_samples, random_state=seed)
high_class_sample = high_class.sample(n=num_samples, random_state=seed)

# Concatenate the sampled data
train_data_sampled = pd.concat([low_class_sample, medium_class_sample, high_class_sample])

# Separate the features and target variable
X_sampled = train_data_sampled
y_sampled = train_labels.loc[train_data_sampled.index]

# Display the count of unique values in the target variable
print(y_sampled.value_counts())

In [ ]:
# do data split
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
# X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_labels, test_size=0.2, random_state=seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=seed)

# Display the shapes of the training and validation sets
print(f"X_train shape: {X_train.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_valid shape: {y_valid.shape}")


In [ ]:
# train random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_model.fit(X_train, y_train)

# Accuracy and F1 score on the training set
train_preds = rf_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_preds)
train_f1 = f1_score(y_train, train_preds, average='macro')

# Accuracy and F1 score on the validation set
valid_preds = rf_model.predict(X_valid)
valid_accuracy = accuracy_score(y_valid, valid_preds)
valid_f1 = f1_score(y_valid, valid_preds, average='macro')

# Display the accuracy and F1 score
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Validation Accuracy: {valid_accuracy:.4f}")
print(f"Validation F1 Score: {valid_f1:.4f}")

In [ ]:
# make predictinos on test data
test_preds = rf_model.predict(test_data)

# convert predictions to original target values
target_mapping = {v: k for k, v in target_mapping.items()}
test_preds = pd.Series(test_preds).map(target_mapping)

# make csv file for submission
submission = pd.DataFrame({
    'UID': test_data.index,
    'Target': test_preds
})

submission.to_csv('submission_v1.csv', index=False)

## Random Forest Full

In [ ]:
# random forest on full data
# Initialize the Random Forest Classifier
rf_model_full = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_model_full.fit(train_data, train_labels)

# Make predictions on the test set
test_preds = rf_model_full.predict(test_data)

# convert predictions to target values
target_mapping_inv = {v: k for k, v in target_mapping.items()}
test_preds = pd.Series(test_preds).map(target_mapping_inv)

# Create a DataFrame with the 'UID' column and the predictions
submission_df = pd.DataFrame({
    'UID': test_data.index,
    'Target': test_preds
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission_4.csv', index=False)

## Naive Bayes

In [ ]:
# use naive bayes
from sklearn.naive_bayes import GaussianNB

# Initialize the Gaussian Naive Bayes Classifier
nb_model = GaussianNB()

# Fit the model
nb_model.fit(X_train, y_train)

# Accuracy and F1 score on the training set
train_preds = nb_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_preds)
train_f1 = f1_score(y_train, train_preds, average='macro')

# Accuracy and F1 score on the validation set
valid_preds = nb_model.predict(X_valid)
valid_accuracy = accuracy_score(y_valid, valid_preds)
valid_f1 = f1_score(y_valid, valid_preds, average='macro')

# Display the accuracy and F1 score
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Validation Accuracy: {valid_accuracy:.4f}")
print(f"Validation F1 Score: {valid_f1:.4f}")

In [ ]:
# get the predictions on test data
test_preds = nb_model.predict(test_data)

# convert predictions to target values
target_mapping_inv = {v: k for k, v in target_mapping.items()}
test_preds = pd.Series(test_preds).map(target_mapping_inv)

# Create a DataFrame with the 'UID' column and the predictions
submission_df = pd.DataFrame({
    'UID': test_data.index,
    'Target': test_preds
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission_5.csv', index=False)

## ensemble models

In [ ]:
# rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)
# gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=seed)
# ada_clf = AdaBoostClassifier(n_estimators=100, random_state=seed)

# # create a voting classifier with soft voting
# ensemble_model = VotingClassifier(
#     estimators=[
#         ('rf', rf_clf),
#         ('gb', gb_clf),
#         ('ada', ada_clf)
#     ],
#     voting='soft'
# )

# # Fit the model
# ensemble_model.fit(X_train, y_train)

# # Make predictions on the validation set
# valid_preds = ensemble_model.predict(X_valid)

# # Calculate the accuracy of the model
# accuracy = accuracy_score(y_valid, valid_preds)

# # Calculate the F1 score of the model
# f1 = f1_score(y_valid, valid_preds, average='macro')
# print(f"F1 Score: {f1}")

# # Display the accuracy of the model
# print(f"Accuracy: {accuracy}")

## Ensemble 2

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

# Initialize classifiers with a random seed
seed = 42
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=seed)
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=seed)
log_reg_clf = LogisticRegression(max_iter=1000, random_state=seed)
svc_clf = SVC(kernel='rbf', probability=True, random_state=seed)  # SVM with soft voting
nb_clf = GaussianNB()

# Create a voting classifier with soft voting
ensemble_model = VotingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('gb', gb_clf),
        ('ada', ada_clf),
        ('log_reg', log_reg_clf),
        ('svc', svc_clf),
        ('nb', nb_clf)
    ],
    voting='soft'
)

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Evaluate on the training set
train_preds = ensemble_model.predict(X_train)
train_f1 = f1_score(y_train, train_preds, average='macro')

# Evaluate on the validation set
valid_preds = ensemble_model.predict(X_valid)
valid_f1 = f1_score(y_valid, valid_preds, average='macro')

# Display the F1 scores
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Validation F1 Score: {valid_f1:.4f}")


In [ ]:
# make predictinos on test data
test_preds = ensemble_model.predict(test_data)

In [ ]:
# print count of unique values in predictions
print(np.unique(test_preds, return_counts=True))

In [ ]:
# convert predictions to original target values
target_mapping = {v: k for k, v in target_mapping.items()}
test_preds = pd.Series(test_preds).map(target_mapping)

In [ ]:
# make csv file for submission
submission = pd.DataFrame({
    'UID': test_data.index,
    'Target': test_preds
})

submission.to_csv('submission_6.csv', index=False)

## XGBoost

In [ ]:
# Use XGBoost

# Initialize the XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, random_state=seed)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
train_preds = xgb_model.predict(X_train)
train_f1 = f1_score(y_train, train_preds, average='macro')
train_accuracy = accuracy_score(y_train, train_preds)

# Make predictions on the validation set
valid_preds = xgb_model.predict(X_valid)
valid_f1 = f1_score(y_valid, valid_preds, average='macro')
valid_accuracy = accuracy_score(y_valid, valid_preds)

# Display the F1 scores
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Validation F1 Score: {valid_f1:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {valid_accuracy:.4f}")

In [ ]:
# ver_1 xgboost
xgb_v1 = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    n_estimators=200,
    max_depth=2,
    random_state=seed
) 

# Fit the model
xgb_v1.fit(X_train, y_train)

# get accuracy and f1 score on training set and validation set
train_preds = xgb_v1.predict(X_train)
train_f1 = f1_score(y_train, train_preds, average='macro')
train_accuracy = accuracy_score(y_train, train_preds)

valid_preds = xgb_v1.predict(X_valid)
valid_f1 = f1_score(y_valid, valid_preds, average='macro')
valid_accuracy = accuracy_score(y_valid, valid_preds)

# Display the F1 scores
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Validation F1 Score: {valid_f1:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {valid_accuracy:.4f}")



## Grid Search

In [ ]:
cv_params = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.1, 0.3, 0.5],
}

csv = GridSearchCV(
    estimator=XGBClassifier(random_state=seed),
    param_grid=cv_params,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1
)

csv.fit(X_train, y_train)

# Get the best parameters
best_params = csv.best_params_

# Display the best parameters
print(best_params)

In [ ]:
cv_params = {
    'max_depth': [ 2, 3, 4, 5, 6, 7],
    'min_child_weight': [1, 2, 3, 4]
}

fixed_params = {
    'n_estimators': 100,
    'learning_rate': 0.1, 
    'random_state': seed
}

csv = GridSearchCV(
    estimator=XGBClassifier(**fixed_params),
    param_grid=cv_params,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1
)

csv.fit(X_train, y_train)

# Get the best parameters
best_params = csv.best_params_

# Display the best parameters
print(best_params)

In [ ]:
cv_params = {
    'subsample': [0.6, 0.8, 0,9, 1.0],
    'max_delta_step': [0, 1, 2, 3, 4]
}

fixed_params = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 3,
    'random_state': seed
}

csv = GridSearchCV(
    estimator=XGBClassifier(**fixed_params),
    param_grid=cv_params,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1
)

csv.fit(X_train, y_train)

# Get the best parameters
best_params = csv.best_params_

# Display the best parameters
print(best_params)

In [ ]:
final_params = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 3,
    'subsample': 0.6,
    'max_delta_step': 1,
    'random_state': seed
}

# Initialize the XGBoost Classifier
xgb_model = XGBClassifier(**final_params)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
train_preds = xgb_model.predict(X_train)
train_f1 = f1_score(y_train, train_preds, average='macro')
train_accuracy = accuracy_score(y_train, train_preds)

# Make predictions on the validation set
valid_preds = xgb_model.predict(X_valid)
valid_f1 = f1_score(y_valid, valid_preds, average='macro')
valid_accuracy = accuracy_score(y_valid, valid_preds)

# Display the F1 scores
print(f"Training F1 Score: {train_f1:.4f}")
print(f"Validation F1 Score: {valid_f1:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {valid_accuracy:.4f}")

In [ ]:
# make predictinos on test data
test_preds = xgb_model.predict(test_data)

# convert predictions to original target values
target_mapping = {v: k for k, v in target_mapping.items()}
test_preds = pd.Series(test_preds).map(target_mapping)

# make csv file for submission
submission = pd.DataFrame({
    'UID': test_data.index,
    'Target': test_preds
})

submission.to_csv('submission_9.csv', index=False)

In [ ]:
# Hyper parameter tuning for XGBoost to get max F1 macro score on validation set 
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}

In [ ]:
def make_predictions(test_fname, predictions_fname):
    # Load the test data
    test_data = pd.read_csv(test_fname)

    predictions = np.array([random.choice([0, 1, 2]) for _ in range(len(test_data))])

    # map 0 -> low, 1 -> medium, 2 -> high
    predictions = np.array(['low', 'medium', 'high'])[predictions]

    # Save the predictions to CSV file containing UID and Target columns
    pd.DataFrame({
        'UID': test_data['UID'],
        'Target': predictions
    }).to_csv(predictions_fname, index=False)


# make_predictions("test.csv", "predictions.csv")